In [1]:
pip install pandas scikit-learn numpy Pillow keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 43.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 46.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 50.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 26.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart th

In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path'])
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit model input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)

Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [3]:
import numpy as np
from keras_tuner import HyperModel, HyperParameters
import keras_tuner as kt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, TimeDistributed, LSTM
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, SGD, RMSprop


def build_model(hp):
    model = Sequential()
    
    # First Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_1_filters', min_value=32, max_value=64, step=16)),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Second Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_2_filters', min_value=64, max_value=128, step=16)),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Third Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_3_filters', min_value=128, max_value=256, step=16)),
                     kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    model.add(TimeDistributed(Flatten()))
    
    # LSTM Layers
    model.add(LSTM(int(hp.Int('lstm_units_1', min_value=80, max_value=120, step=10)), 
                   dropout=hp.Float('dropout_1', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(int(hp.Int('lstm_units_2', min_value=50, max_value=70, step=10)), 
                   dropout=hp.Float('dropout_2', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(int(hp.Int('lstm_units_3', min_value=20, max_value=40, step=10)), 
                   dropout=0.2, 
                   return_sequences=False))
    
    # Fully Connected Layers
    model.add(Dense(int(hp.Int('dense_1_units', min_value=128, max_value=512, step=64)), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.3, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(int(hp.Int('dense_2_units', min_value=64, max_value=256, step=64)), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.4, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(3, activation='softmax'))
    
    # Optimizer
    learning_rate_params = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer_params = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'Adadelta', 'Adagrad'])
    
    if optimizer_params == 'adam':
        optimizer = Adam(learning_rate=learning_rate_params)
    elif optimizer_params == 'sgd':
        optimizer = SGD(learning_rate=learning_rate_params)
    elif optimizer_params == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adagrad':
        optimizer = Adagrad(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adadelta':
        optimizer = Adadelta(learning_rate=learning_rate_params)
    else:
        raise ValueError(f"Invalid optimizer choice: {optimizer_choice}")
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


2024-12-22 15:52:57.313854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
pip install py-cpuinfo


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cpuinfo

info = cpuinfo.get_cpu_info()
print("CPU Flags:", info.get('flags', []))

# Check for AVX2 and FMA
if 'avx2' in info['flags'] and 'fma' in info['flags']:
    print("Your CPU supports AVX2 and FMA!")
else:
    print("AVX2 or FMA support is missing.")


CPU Flags: ['3dnowext', '3dnowprefetch', 'abm', 'adx', 'aes', 'amd_ppin', 'aperfmperf', 'apic', 'arat', 'avic', 'avx', 'avx2', 'bmi1', 'bmi2', 'bpext', 'cat_l3', 'cdp_l3', 'clflush', 'clflushopt', 'clwb', 'clzero', 'cmov', 'cmp_legacy', 'constant_tsc', 'cpb', 'cpuid', 'cqm', 'cqm_llc', 'cqm_mbm_local', 'cqm_mbm_total', 'cqm_occup_llc', 'cr8_legacy', 'cx16', 'cx8', 'dbx', 'de', 'decodeassists', 'extapic', 'extd_apicid', 'f16c', 'flushbyasid', 'fma', 'fpu', 'fsgsbase', 'fxsr', 'fxsr_opt', 'ht', 'hw_pstate', 'ibpb', 'ibrs', 'ibs', 'irperf', 'lahf_lm', 'lbrv', 'lm', 'mba', 'mca', 'mce', 'misalignsse', 'mmx', 'mmxext', 'monitor', 'movbe', 'msr', 'mtrr', 'mwaitx', 'nonstop_tsc', 'nopl', 'npt', 'nrip_save', 'nx', 'osvw', 'osxsave', 'overflow_recov', 'pae', 'pat', 'pausefilter', 'pci_l2i', 'pclmulqdq', 'pdpe1gb', 'perfctr_core', 'perfctr_llc', 'perfctr_nb', 'pfthreshold', 'pge', 'pni', 'popcnt', 'pqe', 'pqm', 'pse', 'pse36', 'rapl', 'rdpid', 'rdpru', 'rdrand', 'rdrnd', 'rdseed', 'rdt_a', 'rdts

In [7]:
pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 3.2 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 32.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 32.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.8/391.8 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.7
    Uninstalling flatbuffers-24.3.7:
      Successfully uninstalled flatbuffers-24.3.7
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: h5py
    Found existing installation: h5py

In [8]:
import numpy as np
import random

# EHO Parameters
num_clans = 1  # Since population size is 5, let's use 1 clan with 5 elephants for simplicity
num_elephants_per_clan = 5
max_generations = 5
epochs = 10  # Number of training epochs

# Boundaries for the hyperparameters
bounds = {
    'conv_1_filters': [32, 64],
    'conv_2_filters': [64, 128],
    'conv_3_filters': [128, 256],
    'lstm_units_1': [80, 120],
    'lstm_units_2': [50, 70],
    'lstm_units_3': [20, 40],
    'dropout_1': [0.2, 0.3],
    'dropout_2': [0.2, 0.3],
    'dropout_3': [0.3, 0.5],
    'dropout_4': [0.2, 0.4],
    'dense_1_units': [128, 512],
    'dense_2_units': [64, 256],
    'learning_rate': [1e-4, 1e-2]
}

# Clan structure
def initialize_population():
    population = []
    for _ in range(num_clans):
        clan = []
        for _ in range(num_elephants_per_clan):
            elephant = {param: random.uniform(bounds[param][0], bounds[param][1]) for param in bounds}
            clan.append(elephant)
        population.append(clan)
    return population

# Fitness evaluation (e.g., model accuracy)
def evaluate_fitness(elephant):
    # Define hyperparameters using the current elephant's values
    hp = HyperParameters()
    for param, value in elephant.items():
        hp.Fixed(param, value)
    
    # Build and compile the model
    model = build_model(hp)
    
    # Train the model
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia, 
        validation_data=(X_val_ischaemia, y_val_ischaemia), 
        epochs=epochs,  # Set epochs to 10
        verbose=0  # Suppress detailed logs
    )
    
    # Retrieve the final epoch's metrics
    training_accuracy = history.history['accuracy'][-1]
    training_loss = history.history['loss'][-1]
    validation_accuracy = history.history['val_accuracy'][-1]
    validation_loss = history.history['val_loss'][-1]
    
    # Print the training and validation metrics
    print(f"Training Accuracy: {training_accuracy}, Training Loss: {training_loss}")
    print(f"Validation Accuracy: {validation_accuracy}, Validation Loss: {validation_loss}")
    
    # Return validation accuracy as fitness score
    return validation_accuracy

# Clan updating operator
def clan_update(clan, matriarch):
    for elephant in clan:
        for param in bounds:
            beta = np.random.uniform(0, 1)
            elephant[param] = beta * matriarch[param] + (1 - beta) * elephant[param]
    return clan

# Separating operator
def separating_operator(clan):
    worst_elephant = sorted(clan, key=evaluate_fitness)[0]
    for param in worst_elephant:
        worst_elephant[param] = random.uniform(bounds[param][0], bounds[param][1])
    return clan

# EHO main loop
def eho_optimization():
    population = initialize_population()
    
    best_solution = None
    best_fitness = -float('inf')
    
    for generation in range(max_generations):
        for clan in population:
            # Find the matriarch (best elephant)
            matriarch = max(clan, key=evaluate_fitness)
            # Update clan members
            clan = clan_update(clan, matriarch)
            # Perform separating operation on worst elephant
            clan = separating_operator(clan)
        
        # Find the best solution in the current generation
        current_best = max([max(clan, key=evaluate_fitness) for clan in population], key=evaluate_fitness)
        current_best_fitness = evaluate_fitness(current_best)
        
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_solution = current_best
        
        print(f"Generation {generation+1}, Best Fitness: {current_best_fitness}")
    
    return best_solution

# Run the EHO optimization
best_hyperparameters = eho_optimization()

# Print the best hyperparameters
print("Best Hyperparameters after EHO Optimization:")
print(best_hyperparameters)

# Evaluate the best model with test accuracy
def evaluate_best_hyperparameters(best_hyperparameters):
    # Define the best hyperparameters using the final best solution
    hp = HyperParameters()
    for param, value in best_hyperparameters.items():
        hp.Fixed(param, value)
    
    # Build and compile the model with best hyperparameters
    model = build_model(hp)
    
    # Train the model with best hyperparameters
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia, 
        validation_data=(X_val_ischaemia, y_val_ischaemia), 
        epochs=epochs,  # Keep it consistent with 10 epochs
        verbose=1  # Show detailed logs now
    )
    
    # Print the final metrics for the best hyperparameters
    training_accuracy = history.history['accuracy'][-1]
    training_loss = history.history['loss'][-1]
    validation_accuracy = history.history['val_accuracy'][-1]
    validation_loss = history.history['val_loss'][-1]
    
    print(f"Best Model Training Accuracy: {training_accuracy}, Training Loss: {training_loss}")
    print(f"Best Model Validation Accuracy: {validation_accuracy}, Validation Loss: {validation_loss}")
    
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test_ischaemia, y_test_ischaemia, verbose=1)
    print(f"Best Model Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")

# Evaluate the best model
evaluate_best_hyperparameters(best_hyperparameters)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  kernel_regularizer=None,
2024-12-22 15:57:01.807113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22066 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-12-22 15:57:01.810596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22066 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:21:00.0, compute capability: 8.9
2024-12-22 15:57:20.979921: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


Training Accuracy: 0.856274425983429, Training Loss: 0.34269315004348755
Validation Accuracy: 0.8540540337562561, Validation Loss: 0.2975222170352936
Training Accuracy: 0.8768272995948792, Training Loss: 0.28716257214546204
Validation Accuracy: 0.8936936855316162, Validation Loss: 0.23921506106853485
Training Accuracy: 0.8218265771865845, Training Loss: 0.40631282329559326
Validation Accuracy: 0.8103603720664978, Validation Loss: 0.42306920886039734
Training Accuracy: 0.8779852390289307, Training Loss: 0.3022981286048889
Validation Accuracy: 0.8738738894462585, Validation Loss: 0.30445581674575806
Training Accuracy: 0.8640903234481812, Training Loss: 0.3175891935825348
Validation Accuracy: 0.8031531572341919, Validation Loss: 0.389813631772995
Training Accuracy: 0.8606165647506714, Training Loss: 0.32313671708106995
Validation Accuracy: 0.8869369626045227, Validation Loss: 0.2781726121902466
Training Accuracy: 0.875090479850769, Training Loss: 0.30227789282798767
Validation Accuracy: 0

In [10]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate metrics for the best hyperparameters
def evaluate_best_hyperparameters(best_hyperparameters):
    # Define the best hyperparameters using the final best solution
    hp = HyperParameters()
    for param, value in best_hyperparameters.items():
        hp.Fixed(param, value)
    
    # Build and compile the model with best hyperparameters
    model = build_model(hp)
    
    # Start training
    start_training_time = time.time()
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia,
        validation_data=(X_val_ischaemia, y_val_ischaemia),
        epochs=10,  # Keep it consistent with the notebook's configuration
        verbose=1  # Show detailed logs
    )
    end_training_time = time.time()
    training_time = end_training_time - start_training_time
    
    # Collect training metrics
    train_accuracy = history.history['accuracy'][-1]
    train_loss = history.history['loss'][-1]
    y_train_pred = np.argmax(model.predict(X_train_ischaemia), axis=1)
    train_f1 = f1_score(y_train_ischaemia, y_train_pred, average='macro')
    train_precision = precision_score(y_train_ischaemia, y_train_pred, average='macro')
    train_recall = recall_score(y_train_ischaemia, y_train_pred, average='macro')
    
    # Validation metrics
    start_validation_time = time.time()
    val_loss, val_accuracy = model.evaluate(X_val_ischaemia, y_val_ischaemia)
    end_validation_time = time.time()
    validation_time = end_validation_time - start_validation_time
    
    y_val_pred = np.argmax(model.predict(X_val_ischaemia), axis=1)
    val_f1 = f1_score(y_val_ischaemia, y_val_pred, average='macro')
    val_precision = precision_score(y_val_ischaemia, y_val_pred, average='macro')
    val_recall = recall_score(y_val_ischaemia, y_val_pred, average='macro')
    
    # Testing metrics
    start_testing_time = time.time()
    test_accuracy = model.evaluate(X_test_ischaemia, y_test_ischaemia, verbose=0)[1]
    y_test_pred = model.predict(X_test_ischaemia)
    y_test_classes = np.argmax(y_test_pred, axis=1)
    end_testing_time = time.time()
    testing_time = end_testing_time - start_testing_time
    
    test_f1 = f1_score(y_test_ischaemia, y_test_classes, average='macro')
    test_precision = precision_score(y_test_ischaemia, y_test_classes, average='macro')
    test_recall = recall_score(y_test_ischaemia, y_test_classes, average='macro')
    
    # Collect metrics in a dictionary
    metrics = {
        'Training Accuracy': train_accuracy,
        'Training Loss': train_loss,
        'Training F1 Score': train_f1,
        'Training Precision': train_precision,
        'Training Recall': train_recall,
        'Training Time (s)': training_time,
        'Validation Accuracy': val_accuracy,
        'Validation Loss': val_loss,
        'Validation F1 Score': val_f1,
        'Validation Precision': val_precision,
        'Validation Recall': val_recall,
        'Validation Time (s)': validation_time,
        'Test Accuracy': test_accuracy,
        'Test F1 Score': test_f1,
        'Test Precision': test_precision,
        'Test Recall': test_recall,
        'Testing Time (s)': testing_time
    }
    
    # Print the metrics
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

# Evaluate the best hyperparameters after EHO optimization
evaluate_best_hyperparameters(best_hyperparameters)



Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 47s 187ms/step - accuracy: 0.6776 - loss: 0.8280 - val_accuracy: 0.7874 - val_loss: 0.4809
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 179ms/step - accuracy: 0.8174 - loss: 0.4129 - val_accuracy: 0.8532 - val_loss: 0.3696
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.8331 - loss: 0.3883 - val_accuracy: 0.8315 - val_loss: 0.4367
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.8375 - loss: 0.3774 - val_accuracy: 0.8581 - val_loss: 0.2837
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 178ms/step - accuracy: 0.8468 - loss: 0.3472 - val_accuracy: 0.8716 - val_loss: 0.2892
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 174ms/step - accuracy: 0.8467 - loss: 0.3496 - val_accuracy: 0.8302 - val_loss: 0.3732
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 178ms/step - accuracy: 0.8714 - loss: 0.3007 - val_accuracy: 0.8473 - val_loss: 0.3274
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.8750 - loss: 0

In [11]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate metrics for the best hyperparameters
def evaluate_best_hyperparameters(best_hyperparameters):
    # Define the best hyperparameters using the final best solution
    hp = HyperParameters()
    for param, value in best_hyperparameters.items():
        hp.Fixed(param, value)
    
    # Build and compile the model with best hyperparameters
    model = build_model(hp)
    
    # Start training
    start_training_time = time.time()
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia,
        validation_data=(X_val_ischaemia, y_val_ischaemia),
        epochs=50,
        verbose=1 
    )
    end_training_time = time.time()
    training_time = end_training_time - start_training_time
    
    # Collect training metrics
    train_accuracy = history.history['accuracy'][-1]
    train_loss = history.history['loss'][-1]
    y_train_pred = np.argmax(model.predict(X_train_ischaemia), axis=1)
    train_f1 = f1_score(y_train_ischaemia, y_train_pred, average='macro')
    train_precision = precision_score(y_train_ischaemia, y_train_pred, average='macro')
    train_recall = recall_score(y_train_ischaemia, y_train_pred, average='macro')
    
    # Validation metrics
    start_validation_time = time.time()
    val_loss, val_accuracy = model.evaluate(X_val_ischaemia, y_val_ischaemia)
    end_validation_time = time.time()
    validation_time = end_validation_time - start_validation_time
    
    y_val_pred = np.argmax(model.predict(X_val_ischaemia), axis=1)
    val_f1 = f1_score(y_val_ischaemia, y_val_pred, average='macro')
    val_precision = precision_score(y_val_ischaemia, y_val_pred, average='macro')
    val_recall = recall_score(y_val_ischaemia, y_val_pred, average='macro')
    
    # Testing metrics
    start_testing_time = time.time()
    test_accuracy = model.evaluate(X_test_ischaemia, y_test_ischaemia, verbose=0)[1]
    y_test_pred = model.predict(X_test_ischaemia)
    y_test_classes = np.argmax(y_test_pred, axis=1)
    end_testing_time = time.time()
    testing_time = end_testing_time - start_testing_time
    
    test_f1 = f1_score(y_test_ischaemia, y_test_classes, average='macro')
    test_precision = precision_score(y_test_ischaemia, y_test_classes, average='macro')
    test_recall = recall_score(y_test_ischaemia, y_test_classes, average='macro')
    
    # Collect metrics in a dictionary
    metrics = {
        'Training Accuracy': train_accuracy,
        'Training Loss': train_loss,
        'Training F1 Score': train_f1,
        'Training Precision': train_precision,
        'Training Recall': train_recall,
        'Training Time (s)': training_time,
        'Validation Accuracy': val_accuracy,
        'Validation Loss': val_loss,
        'Validation F1 Score': val_f1,
        'Validation Precision': val_precision,
        'Validation Recall': val_recall,
        'Validation Time (s)': validation_time,
        'Test Accuracy': test_accuracy,
        'Test F1 Score': test_f1,
        'Test Precision': test_precision,
        'Test Recall': test_recall,
        'Testing Time (s)': testing_time
    }
    
    # Print the metrics
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

# Evaluate the best hyperparameters after EHO optimization
evaluate_best_hyperparameters(best_hyperparameters)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  kernel_regularizer=None,


Epoch 1/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 47s 190ms/step - accuracy: 0.7036 - loss: 0.8219 - val_accuracy: 0.8023 - val_loss: 0.4913
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 179ms/step - accuracy: 0.8202 - loss: 0.4003 - val_accuracy: 0.8081 - val_loss: 0.4043
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.8401 - loss: 0.3689 - val_accuracy: 0.8671 - val_loss: 0.3397
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 38s 178ms/step - accuracy: 0.8346 - loss: 0.3665 - val_accuracy: 0.8514 - val_loss: 0.3230
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 180ms/step - accuracy: 0.8574 - loss: 0.3237 - val_accuracy: 0.7977 - val_loss: 0.6633
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 180ms/step - accuracy: 0.8599 - loss: 0.3354 - val_accuracy: 0.8631 - val_loss: 0.3067
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 178ms/step - accuracy: 0.8696 - loss: 0.3066 - val_accuracy: 0.8054 - val_loss: 0.4038
Epoch 8/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 39s 180ms/step - accuracy: 0.8686 - loss: 0